# Un notebook pour filtrer les données

In [145]:
import random
import pandas as pd
import plotly.express as px

### Garder les stations qui sont dans les zones éligibles

In [146]:
df = pd.read_csv("s3://jedha-final-project-jrat/zones.csv")

In [147]:
df.head()

,Y,X,Latitude,Longitude,Color
0,79,571,50.198832,3.355932,"rgb(206,136,255)"
1,80,571,50.188113,3.355932,"rgb(206,136,255)"
2,80,579,50.188113,3.480975,"rgb(208,142,255)"
3,81,572,50.177394,3.371562,"rgb(206,136,255)"
4,83,577,50.155957,3.449714,"rgb(208,142,255)"


In [148]:
def is_in_zone(coords: tuple[float, float],
               zone: list[tuple[float, float]]) -> bool:
    """Verify if a place is in a zone

    Args:
        coords (tuple[float, float]): position station meteo ou parc eolien
        zone (list[tuple[float, float]]): zone d'autorisation

    Returns:
        bool

    Example:
        coords = (43.02, 1.05)
        zone = [(50.1, -3), (48.3, 5), (32.5, -1), (25.2, 3), (40.0, 2.5)]
        result = is_in_zone(coords, zone)
        print(result)
    """
    x, y = coords
    n = len(zone)
    inside = False

    p1x, p1y = zone[0]
    for i in range(n + 1):
        p2x, p2y = zone[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

In [149]:
zone = df[["Latitude", "Longitude"]]
zone

,Latitude,Longitude
0,50.198832,3.355932
1,50.188113,3.355932
2,50.188113,3.480975
3,50.177394,3.371562
4,50.155957,3.449714
...,...,...
7799,42.567189,9.092288
7800,42.567189,9.295484
7801,42.556470,9.107919
7802,42.492159,9.076658


In [150]:
coordinates = [(random.uniform(40, 70), random.uniform(-10, 10)) for _ in range(100)]
zone = list(zone.itertuples(index=False, name=None))

In [151]:
fig = px.scatter_mapbox(zone, lat=0, lon=1,
                        zoom=5,
                        height=800)

In [152]:
for place in coordinates:
    result = is_in_zone(place, zone)

    if result:
        color = "green"
    else:
        color = "red"

    fig.add_scattermapbox(lat=[place[0]], lon=[place[1]],
                        mode="markers",
                        marker=dict(size=10, color=color))

In [153]:
fig.update_layout(mapbox_style="open-street-map")

fig.show()